# Problem setting

- 1 vehicle
- Nodes: Depot, 5 Pickup locations (P1 to P5), and 5 Delivery locations (D1 to D5).
- Time Windows: Randomly generated for each pickup and delivery location.
- Stack Limit: 3 (maximum number of items the vehicle can carry).
- Cost Matrix: Randomly generated costs between each pair of locations.

## Gurobi

In [2]:
using JuMP, Gurobi, CPLEX

In [17]:
# Problem dimensions
num_pickups = 5
num_deliveries = 5
num_nodes = 1 + num_pickups + num_deliveries # Including depot
vehicle_capacity = 3

3

In [18]:
# Random cost matrix (symmetric for simplicity)
cost_matrix = rand(1:10, num_nodes, num_nodes)
for i in 1:num_nodes
    cost_matrix[i,i] = 0
end

cost_matrix

11×11 Matrix{Int64}:
  0   7  2   7   2  10   5   3  5   6   9
 10   0  5   7  10   9   7   6  5   4  10
  7   9  0   4   4   8   8   7  1   8   4
 10   7  4   0   5   6   8   8  4   6   6
 10   7  8  10   0   5   8   6  7  10   3
  9  10  3   7   7   0  10   9  2   2   7
 10   3  6   7   3   4   0   7  5  10   9
  8   3  3   1   9   5   6   0  1   7   8
  8   1  8   8   9   2   5   1  0   9   6
  9   5  3   4   3   9   7  10  1   0   6
  7   5  7   6   4   3   5   6  8   4   0

In [19]:
# Random service durations at each location
service_durations = rand(1:4, num_nodes) # Service durations between 1 to 4 units

service_durations

11-element Vector{Int64}:
 3
 2
 4
 3
 1
 3
 4
 2
 3
 4
 4

In [20]:
# Generate random time windows
function generate_random_time_window()
    start_time = rand(0:16)  # Start times between 0 and 16
    end_time = start_time + rand(1:8)  # Duration between 1 and 8 hours
    return (start_time, min(end_time, 24))  # Ensure end time doesn't exceed 24
end

# Time windows for each node, including depot
time_windows = [(0, 24)]  # Depot is always open
for _ in 1:(num_pickups + num_deliveries)
    push!(time_windows, generate_random_time_window())
end

time_windows

11-element Vector{Tuple{Int64, Int64}}:
 (0, 24)
 (1, 5)
 (7, 15)
 (11, 16)
 (9, 14)
 (1, 4)
 (3, 7)
 (6, 11)
 (5, 13)
 (16, 17)
 (0, 7)

In [21]:
# Random travel times between each pair of locations
travel_time_matrix = rand(5:15, num_nodes, num_nodes) # Travel times between 5 to 15 units
for i in 1:num_nodes
    travel_time_matrix[i,i] = 0
end

travel_time_matrix

11×11 Matrix{Int64}:
  0  12   8   8   5  14   9  15   7  14  13
  5   0  14  15  11   5  11   6  11  12   6
 13   9   0  12   9  15  13   7   7   6  15
  7   6   7   0   6  12  15   6   8  15  10
 14   7  15   9   0   8   6   7  15   6   8
  6  14   8  14  12   0  15   6  14  10  11
 13   9   9  14   7   5   0  11  15  14  15
  8   9   9   9  10   5  11   0   8  15   5
 12   7  12  11  10  15   7  13   0   6  12
 14  14   7  10   9   6   8  15  12   0  13
  6  14  13  11  14  14   8  10  15  15   0

In [22]:
M = 10000

# 모델
model = Model(CPLEX.Optimizer)

# 변수
@variable(model, x[1:num_nodes, 1:num_nodes], Bin)
@variable(model, service_start[1:num_nodes])
@variable(model, vehicle_load[1:num_nodes], Int)

# `M` 값 설정 - 충분히 큰 값으로 설정
M = 10000

# 목적 함수: 총 비용 최소화
@objective(model, Min, sum(cost_matrix[i,j] * x[i,j] for i in 1:num_nodes, j in 1:num_nodes))

# 제약 조건
# 1. 각 픽업 및 배달은 정확히 한 번씩 방문
for j in 2:num_nodes
    @constraint(model, sum(x[i,j] for i in 1:num_nodes) == 1)
    @constraint(model, sum(x[j,i] for i in 1:num_nodes) == 1)
end

# 2. 유효 경로를 보장하기 위한 흐름 제약 조건
for j in 2:num_nodes
    @constraint(model, sum(x[i,j] for i in 1:num_nodes) - sum(x[j,i] for i in 1:num_nodes) == 0)
end

# 3. 이동 및 서비스 시간을 포함한 시간 창 제약 조건 업데이트
for i in 1:num_nodes
    for j in 1:num_nodes
        if i != j
            @constraint(model, service_start[j] >= service_start[i] + service_durations[i] + travel_time_matrix[i,j] - (1 - x[i,j]) * M)
        end
    end
end

# 4. 차량 용량 제약 조건
for i in 1:num_nodes
    @constraint(model, vehicle_load[i] >= 0)
    @constraint(model, vehicle_load[i] <= vehicle_capacity)
end

# 차량 하중 업데이트 제약 조건
for i in 1:num_nodes
    for j in 1:num_nodes
        if i != j
            # 픽업은 2에서 num_pickups + 1까지이며 배달은 이후 노드
            pickup = 2:num_pickups+1
            delivery = num_pickups+2:num_nodes

            @constraint(model, vehicle_load[j] >= vehicle_load[i] - vehicle_capacity * (1 - x[i,j]))
            if i in pickup
                @constraint(model, vehicle_load[j] >= vehicle_load[i] + x[i,j] - vehicle_capacity * (1 - x[i,j]))
            elseif i in delivery
                @constraint(model, vehicle_load[j] <= vehicle_load[i] - x[i,j])
            end
        end
    end
end

# 자기 자신으로의 이동 금지 제약 조건
for i in 1:num_nodes
    @constraint(model, x[i,i] == 0)
end


# 문제 해결
optimize!(model)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
Tried aggregator 1 time.
MIP Presolve eliminated 83 rows and 11 columns.
MIP Presolve modified 200 coefficients.
Reduced MIP has 290 rows, 132 columns, and 1010 nonzeros.
Reduced MIP has 110 binaries, 11 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.83 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (2.35 ticks)
Parallel b&c, 24 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (2.35 ticks)


In [23]:
# 최적화 상태 확인
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal!")
    println("총 비용: ", objective_value(model))

    # 경로 추출
    println("경로: ")
    for i in 1:num_nodes
        for j in 1:num_nodes
            if value(x[i,j]) > 0.5 # x[i,j]가 1에 가까우면 경로에 포함
                println("노드 $i 에서 노드 $j 로 이동")
            end
        end
    end

    # 서비스 시작 시간 추출
    println("\n서비스 시작 시간: ")
    for i in 1:num_nodes
        println("노드 $i: ", value(service_start[i]))
    end

    # 차량 하중 추출
    println("\n차량 하중: ")
    for i in 1:num_nodes
        println("노드 $i: ", value(vehicle_load[i]))
    end
else
    println("최적의 솔루션을 찾지 못했습니다. 상태: ", status)
end

최적의 솔루션을 찾지 못했습니다. 상태: INFEASIBLE


## Dynmaic Programming